In [1]:
import sqlite3

In [2]:
db=sqlite3.connect("mt_collections.db")
cur= db.cursor()

In [3]:
#uploading collections csv file
cur.execute ('''drop table if exists collections''')
cur.execute('''create table collections(
                                gpo STRING,
                                idn STRING,
                                fa_num INTEGER,
                                fa_name STRING,
                                franchise STRING,
                                product_line TEXT,
                                manufacturer STRING,
                                item_num STRING,
                                item_description STRING,
                                month TEXT,
                                quarter STRING,
                                year STRING,
                                usage INTEGER,
                                collected INTEGER,
                                accepted INTEGER,
                                dnr INTEGER,
                                available INTEGER,
                                purchased INTEGER,
                                cancelled FLOAT)''')

db.commit()

In [4]:
with open ('MT Collections.csv','r') as ifile:
    next(ifile)
    for idx, line in enumerate(ifile.readlines()):
        line = line.strip().split(",")
        
        
        
        
        cur.execute('''insert into collections( gpo,idn, fa_num,fa_name, franchise,product_line, manufacturer, item_num, item_description,
        month, quarter,year,usage, collected, accepted,dnr,available, purchased, cancelled) VALUES (?,?,?,?,?,?,?,?
        ,?,?,?,?,?,?,?,?,?,?,?)''',line)
        
        db.commit()

In [5]:
#uploading the the csv file that records my inservices
cur.execute('''drop table if exists inservice''')

cur.execute('''create table inservice(
                                fa_name STRING,
                                T_EVS FLOAT,
                                q3_2018_evs FLOAT,
                                q4_2018_evs FLOAT,
                                q1_2019_EVS FLOAT,
                                q2_2019_EVS FLOAT,
                                q3_2019_EVS FLOAT,
                                T_OR FLOAT,
                                q3_2018_OR FLOAT,
                                q4_2018_OR FLOAT,
                                q1_2019_OR FLOAT,
                                q2_2019_OR FLOAT,
                                q3_2019_OR FLOAT)
                               
''')

db.commit()

In [6]:
with open('Inservice_final.csv','r')as ifile:
    next(ifile)
    for idx, line in enumerate (ifile.readlines()):
        line= line.strip().split(",")
        
        
        cur.execute(''' insert into inservice(fa_name, T_EVS, q3_2018_evs ,q4_2018_evs,q1_2019_EVS,q2_2019_EVS,q3_2019_EVS,T_OR,
        q3_2018_OR,q4_2018_OR,q1_2019_OR,q2_2019_OR,q3_2019_OR ) 
        values(?,?,?,?,?,?,?,?,?,?,?,?,?)''',line)
        
        db.commit()

In [7]:
# creating a table that displays total 2019 non-invasive collections and evs inservices

cur.execute ('''drop table if exists evs_collections_2019 ''')
cur.execute('''create table evs_collections_2019 as
                                        select fa_name,year as year_2019,franchise,sum(collected)as non_collected_2019
                                        from collections
                                        where franchise="NON" and year='2019'
                                        group by fa_name
                                        order by year''')

db.commit()

In [8]:
#bringing in 2018 collections and combining to one table

cur.execute ('''drop table if exists evs_collections_2018 ''')
cur.execute('''create table evs_collections_2018 as
                                        select fa_name,year as year_2018,franchise,sum(collected)as non_collected_2018 
                                        from collections
                                        where franchise="NON" and year='2018'
                                        group by fa_name
                                        order by year''')

db.commit()

In [9]:
#combining the tables this is manual but this was the only way I could get it to be formatted the way I wanted

cur.execute ('''drop table if exists evs_collections ''')
cur.execute(''' create table evs_collections as
                                select evs_collections_2019.*,evs_collections_2018.*
                                from evs_collections_2019 
                                join evs_collections_2018 on evs_collections_2019.fa_name=evs_collections_2018.fa_name
                                
                                ''')
db.commit()

In [10]:
cur.execute ('''drop table if exists surgical_collections_2019  ''')
cur.execute('''create table surgical_collections_2019 as
                                        select fa_name,year as year_2019,franchise,sum(collected) as surg_collected_2019
                                        from collections
                                        where franchise="SUR" and year='2019'
                                        group by fa_name
                                        ''')

db.commit()

In [11]:
cur.execute ('''drop table if exists surgical_collections_2018  ''')

cur.execute('''create table surgical_collections_2018 as
                                        select fa_name,year as year_2018,franchise,sum(collected) as surg_collected_2018
                                        from collections
                                        where franchise="SUR" and year='2018'
                                        group by fa_name
                                        ''')

db.commit()

In [12]:
cur.execute ('''drop table if exists surgical_collections  ''')


cur.execute(''' create table surgical_collections as
                                select surgical_collections_2019.*,surgical_collections_2018.*
                                from surgical_collections_2019 
                                join surgical_collections_2018 on surgical_collections_2019.fa_name=surgical_collections_2018.fa_name
                                
                                ''')
db.commit()

Now I am going to join the inservice table to the total evs and surgical collections.  Most of my inservices are either in quarter 3 and 4 of 2018 and quarter 1 of 2019.  I want to see which quarters the inservices have the most effect on 2019 collections

In [13]:
cur.execute ('''drop table if exists surgical_collections_ins  ''')


cur.execute('''create table surgical_collections_ins as
                                        select surgical_collections.*,inservice.*
                                        from surgical_collections
                                        join inservice on surgical_collections.fa_name=inservice.fa_name 
                                                        
                                                        ''')
db.commit()

In [14]:
cur.execute ('''drop table if exists evs_collections_ins  ''')

cur.execute('''create table evs_collections_ins as
                                        select evs_collections.*,inservice.*
                                        from evs_collections
                                        join inservice on evs_collections.fa_name=inservice.fa_name 
                                                        
                                                        ''')
db.commit()